# Background information: Running neural networks in Tensorflow's Graph Mode vs Eager Mode

So far you've probably used numpy and append methods to keep track of your model's performance throughout training and validation. 

This comes with several disadvantages. One disadvantage is that your training loop (i.e. iterating over the training dataset for a number of epochs, calling the train_step on each batch, and then doing the same for the validation dataset with the test_step) can not run in *graph mode* which is most often more performant compared to the *eager mode* in Tensorflow.

Tensorflow was originally built to run deep learning models as a static computational graph that, once constructed, is faster compared to a dynamically built graph. To enable your subclassed model to run and train in graph mode, you need to use the tf.function decorator on the function/method you want to run in graph mode (usually the train_step and test_step). 

The advantage of using the eager mode instead is that you are allowed to do anything that Python allows you to do. It also gives you more readable error messages for debugging. You can use lists and append to them, you can set attributes of objects etc. What these have in common is that they are considered as **side-effects** of a function - operations and assignments that change global variables inside a function. **Pure functions** on the other hand do not change any variables, they simply return what they compute, which can then be assigned to variables outside of the function. Read more about the specifics here: https://www.tensorflow.org/guide/function

# Keras metrics

Instead of appending loss values to a list or a numpy array, we usually rely on something more convenient: tf.keras.metrics objects.

Keras comes with useful objects that support tracking a variable, such as the loss over an entire epoch, computing the average over all losses efficiently while allowing for graph mode train and test step functions/methods.


Let's assume we have one **epoch** with a small dataset that fills only **4 batches**, so to compute the average loss for this epoch, we want to average over four loss values. 

We can do this with a tf.keras.metrics.Mean object, which has an **update_state** method, that takes a scalar value to take into account for the running average, a **result** method, to obtain the result, and a **reset_states** method that resets the metric (after an epoch and between the training and validation steps).

To see what is going on we print the metric's result after each batch

In [27]:
import tensorflow as tf

loss_function = tf.keras.losses.MeanSquaredError()

# instantiate metric object (usually in the model's constructor, i.e. __init__, method)
loss_metric = tf.keras.metrics.Mean()

# ITERATING OVER A NUMBER OF EPOCHS:
for e in range(4):

    # ITERATING OVER TRAINING DATA
    for batch in range(1000):

        model_output = tf.random.uniform(shape=(4,1))*10
        target = tf.random.uniform(shape=(4,1))*10

        loss = loss_function(target, model_output)

        loss_metric.update_state(values=loss)

    tf.print(f"Epoch {e}: loss: {loss_metric.result()}")

    # RESETTING THE METRICS
    loss_metric.reset_states()

    # ITERATING OVER VALIDATION DATA
    for batch in range(200):

        model_output = tf.random.uniform(shape=(4,1))*10

        target = tf.random.uniform(shape=(4,1))*10

        val_loss = loss_function(target, model_output)

        loss_metric.update_state(values=val_loss)

    tf.print(f"Epoch {e}: val_loss: {loss_metric.result()} \n")

    # RESETTING THE METRIC BEFORE NEXT EPOCH
    loss_metric.reset_states()

Epoch 0: loss: 17.090959548950195
Epoch 0: val_loss: 15.924022674560547 

Epoch 1: loss: 16.673707962036133
Epoch 1: val_loss: 16.996715545654297 

Epoch 2: loss: 16.762340545654297
Epoch 2: val_loss: 17.6639347076416 

Epoch 3: loss: 17.127212524414062
Epoch 3: val_loss: 16.37676429748535 



## Accuracy metrics for binary classification
As you see we can use tf.keras.metrics.Mean objects to compute running averages without using numpy or list appends. Keras comes with such metric objects for a number of different metrics that we might use to evaluate our model's performance, such as **BinaryAccuracy** in the context of binary classification and **CategoricalAccuracy** **TopKCategoricalAccuracy** in the context of multi-class classification. 

In [38]:
# instantiate metric object (usually in the model's constructor, i.e. __init__, method)
loss_metric = tf.keras.metrics.Mean(name="loss")
binary_accuracy_metric = tf.keras.metrics.BinaryAccuracy(name="accuracy")
loss_function = tf.keras.losses.BinaryCrossentropy()

# What happens in one epoch in the training loop:

# training on train data (4 batches)

for e in range(5):
    for train_batch in range(100):
        target = tf.random.uniform(shape=(4,1),minval=0, maxval=1, dtype=tf.int32)
        model_output = tf.random.uniform(shape=(4,1))

        loss = loss_function(target, model_output)
        # 
        loss_metric.update_state(values=loss)
        binary_accuracy_metric.update_state(target, model_output)
    tf.print(f"Epoch {e}: loss: {loss_metric.result()}")
    tf.print(f"Epoch {e}: accuracy: {binary_accuracy_metric.result()}")

    # RESETTING METRICS BEFORE EVALUATION
    loss_metric.reset_states()

    for val_batch in range(20):
        target = tf.random.uniform(shape=(4,1),minval=0, maxval=1, dtype=tf.int32)
        model_output = tf.random.uniform(shape=(4,1))

        loss = loss_function(target, model_output)
        loss_metric.update_state(values=loss)
        binary_accuracy_metric.update_state(target, model_output)

    tf.print(f"Epoch {e}: val_loss: {loss_metric.result()}")
    tf.print(f"Epoch {e}: val_accuracy: {binary_accuracy_metric.result()} \n")

# resetting the metric before the next epoch
loss_metric.reset_states()



Epoch 0: loss: 0.9985584020614624
Epoch 0: accuracy: 0.5024999976158142
Epoch 0: val_loss: 1.083017110824585
Epoch 0: val_accuracy: 0.4958333373069763 

Epoch 1: loss: 0.9886395335197449
Epoch 1: accuracy: 0.4909090995788574
Epoch 1: val_loss: 1.0901201963424683
Epoch 1: val_accuracy: 0.49270832538604736 

Epoch 2: loss: 1.029697299003601
Epoch 2: accuracy: 0.4904411733150482
Epoch 2: val_loss: 0.9431459307670593
Epoch 2: val_accuracy: 0.4902777671813965 

Epoch 3: loss: 0.9510965943336487
Epoch 3: accuracy: 0.48750001192092896
Epoch 3: val_loss: 0.9767802357673645
Epoch 3: val_accuracy: 0.4885416626930237 

Epoch 4: loss: 0.9949010014533997
Epoch 4: accuracy: 0.49698275327682495
Epoch 4: val_loss: 1.069822907447815
Epoch 4: val_accuracy: 0.49541667103767395 



## Accuracy metrics for multi-class categorization tasks

In multi-class classification, we use the **CategoricalCrossentropy** as our loss function. To track the accuracy, we need to use a different keras metric, **CategoricalAccuracy**

In [55]:
# instantiate metric object (usually in the model's constructor, i.e. __init__, method)
loss_metric = tf.keras.metrics.Mean(name="loss")
accuracy_metric = tf.keras.metrics.CategoricalAccuracy(name="accuracy")
loss_function = tf.keras.losses.CategoricalCrossentropy()

for e in range(5):
    for train_batch in range(100):
        
        # create random one-hot targets (labels)
        target = tf.one_hot(tf.random.uniform(minval=0,maxval=9, shape=(4,),dtype=tf.int32), depth=10)

        # create random model output (for each element in the batch the values are probabilities that sum to 1)
        model_output = tf.nn.softmax(tf.random.uniform(shape=(4,10)), axis=-1)

        loss = loss_function(target, model_output)

        loss_metric.update_state(values=loss)
        accuracy_metric.update_state(target, model_output)
    tf.print(f"Epoch {e}: loss: {loss_metric.result()}")
    tf.print(f"Epoch {e}: accuracy: {accuracy_metric.result()}")

    # RESETTING METRICS BEFORE EVALUATION
    loss_metric.reset_states()

    for val_batch in range(20):
        target = tf.one_hot(tf.random.uniform(minval=0,maxval=9, shape=(4,),dtype=tf.int32), depth=10)
        model_output = tf.nn.softmax(tf.random.uniform(shape=(4,10)), axis=-1)

        loss = loss_function(target, model_output)
        loss_metric.update_state(values=loss)
        accuracy_metric.update_state(target, model_output)

    tf.print(f"Epoch {e}: val_loss: {loss_metric.result()}")
    tf.print(f"Epoch {e}: val_accuracy: {accuracy_metric.result()} \n")

    # resetting the metric before the next epoch
    loss_metric.reset_states()



Epoch 0: loss: 2.3434877395629883
Epoch 0: accuracy: 0.09749999642372131
Epoch 0: val_loss: 2.368985414505005
Epoch 0: val_accuracy: 0.1041666641831398 

Epoch 1: loss: 2.3544280529022217
Epoch 1: accuracy: 0.09772727638483047
Epoch 1: val_loss: 2.3128294944763184
Epoch 1: val_accuracy: 0.10208333283662796 

Epoch 2: loss: 2.337564468383789
Epoch 2: accuracy: 0.1044117659330368
Epoch 2: val_loss: 2.330190896987915
Epoch 2: val_accuracy: 0.10486111044883728 

Epoch 3: loss: 2.338282823562622
Epoch 3: accuracy: 0.10326086729764938
Epoch 3: val_loss: 2.3272953033447266
Epoch 3: val_accuracy: 0.10520832985639572 

Epoch 4: loss: 2.3113150596618652
Epoch 4: accuracy: 0.10517241060733795
Epoch 4: val_loss: 2.3447346687316895
Epoch 4: val_accuracy: 0.1041666641831398 



## Conclusion

- We can use keras metrics instead of tedious numpy loss tracking and metric computations. 

- Keras metrics can be used with Tensorflow's graph mode, while list appends and numpy operations generally can not

- The convenient compile and fit methods of the tf.keras.Model class use keras metrics under the hood. 
    - Starting to use the metric objects gets us a step closer to being able to use these tools.



Next: **Tensorboard for logging**, log all possible kinds of training data to the tensorboard